# Music Recommendation System using Spotify API

![](https://www.protocol.com/media-library/spotify-app-with-headphones-on.png?id=29171621&width=1000&height=750&quality=85&coordinates=0%2C0%2C0%2C0)

## Recommendation systems
**Recommendation systems** are algorithms and techniques designed to provide personalized suggestions or recommendations to users. These systems analyze user data and item attributes to generate relevant and tailored recommendations. Recommendation systems are widely used in various applications such as e-commerce (product recommendations), streaming services (movie/music recommendations), news websites (article recommendations), and more to enhance user experiences and drive engagement. 

In this notebook, I will be demonstrating one such application of Recommendation systems by building a Music Recommendation System using the Spotify API. The music recommendation system will assist users in discovering new and relevant musical content based on their preferences and listening behaviour.

### Table of Contents
1. Introduction
2. Spotify API

## Spotify API
**Spotify API** (Application Programming Interface) is a set of web services and tools provided by Spotify that allows developers to interact with the Spotify platform and access various music-related data and functionalities. Developers can use the Spotify API to integrate Spotify's vast music catalog, user playlists, and user-related data into their own applications, websites, or services.

I will be using the Spotify API for requesting and collecting real-time music data from Spotify. For accessing the API you first have to sign up for a developer account on Spotify. As I have already done this I am skipping this step. For more information you can check out the [Spotify API documentation](https://developer.spotify.com/documentation/web-api).

After completing the preliminary steps now we come to the coding part.

In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()